In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
date = "2022-01-20"

In [3]:
df_contest = pd.read_csv("contest_data_" + date + ".csv")
df_user_profile = pd.read_csv("user_profile_" + date + ".csv")

In [4]:
df_contest.head()

,id,title,url,seeker,start_time,name_to_incorporate,slogan_to_incorporate,description,industry,colors_to_explore,...,other_notes,entry_count,deleted_entry_count,deleted,withdrawn,declined,prize,entries,deleted_entries,winners
0,1,Looking for a sleek and creative logo design f...,https://99designs.hk/logo-design/contests/look...,demetrilinton,"Tue, 18 Jan 2022 10:48:02 +0000",VP,ShotByVP,We are multimedia company specializing in phot...,Photography,['Designers choose'],...,"The logo can be ""shotbyvp"" or just a visually ...",92,91,0,91,0,US$190,"[{'entry_id': 'entry-177', 'participant_id': '...","[{'entry_id': 'entry-130', 'participant_id': '...",['entry-86']
1,2,Logo Needed to Make People Think of San Antoni...,https://99designs.hk/logo-design/contests/logo...,courtneH,"Mon, 17 Jan 2022 18:40:13 +0000",Unspecified,Coming Home in San Antone,Real Estate Sales and Local Blogging. Home Buy...,Real Estate & Mortgage,['Unspecified'],...,"Somehow, the Redbird Logo below must be incorp...",8,25,0,25,0,US$190,"[{'entry_id': 'entry-30', 'participant_id': '3...","[{'entry_id': 'entry-13', 'participant_id': '3...",['entry-30']
2,3,Real Estate Team Logo,https://99designs.hk/logo-design/contests/real...,quincw,"Mon, 17 Jan 2022 17:05:49 +0000",Hieber Realtor Group,Unspecified,Real Estate. Home owners and potential buyers,Real Estate & Mortgage,['Designers choose'],...,Unspecified,20,12,0,12,0,US$190,"[{'entry_id': 'entry-29', 'participant_id': '1...","[{'entry_id': 'entry-19', 'participant_id': '1...",['entry-29']
3,4,Design for women lawyers who are powerful lead...,https://99designs.hk/logo-design/contests/desi...,lexleb,"Mon, 17 Jan 2022 01:06:20 +0000",Luminous Warriors - Mind Over Law,Unspecified,Lawyers -----women lawyers who are leaders,Business & Consulting,"['Purples', 'Blues', 'Aquas']",...,"I’m mostly interested in combination logos, bu...",40,8,0,8,0,US$471,"[{'entry_id': 'entry-37', 'participant_id': '1...","[{'entry_id': 'entry-36', 'participant_id': '2...",['entry-37']
4,5,Logo für shopynator.de,https://99designs.hk/logo-design/contests/logo...,mbzdega,"Sun, 16 Jan 2022 18:33:32 +0000",shopynator,we empower local business,"Wir haben es uns zur Aufgabe gemacht, den loka...",Business & Consulting,['Designers choose'],...,Ich bevorzuge ein rundes oder quadratisches Lo...,4,22,0,11,11,US$195,"[{'entry_id': 'entry-24', 'participant_id': '1...","[{'entry_id': 'entry-25', 'participant_id': '1...",['entry-24']


In [5]:
# get user id
def get_user_level(row):

    res = []
    entries = row['entries']
    res += re.findall("'participant_id': '\d{5,8}'", entries)
    
    deleted_entries = row['deleted_entries']
    res += re.findall("'participant_id': '\d{5,8}'", deleted_entries)

    res = list(set(res))

    # search participant ids
    ids = list(map(lambda x: re.search("\d{5,8}", x).group(0), res))

    # get user levels
    entry_level = 0
    mid_level = 0
    top_level = 0

    for id in ids:
        try:
            certification = df_user_profile[df_user_profile["user_id"] == int(id)]["certification"].values[0]
            # print(certification)
        except:
            certification = []
        if 'Top Level' in certification:
            top_level += 1
        elif 'Mid Level' in certification:
            mid_level += 1
        else:
            entry_level += 1

    # get top_n participant's contest won and runner up
    contest_won = []
    runner_up = []
    for id in ids:
        try:
            contest_won.append(df_user_profile[df_user_profile["user_id"] == int(id)]["contests_won"].values[0])
            runner_up.append(df_user_profile[df_user_profile["user_id"] == int(id)]["runner_up"].values[0])
            # print(certification)
        except:
            pass

    # sort contest won and runner up in descending order
    contest_won.sort(reverse=True)
    runner_up.sort(reverse=True)

    if len(contest_won) >= 10:
        avg_contest_won = np.mean(contest_won)
        avg_contest_won_top_10 = np.mean(contest_won[:10])
        avg_contest_won_top_5 = np.mean(contest_won[:5])
    elif len(contest_won) >= 5:
        avg_contest_won = np.mean(contest_won)
        avg_contest_won_top_10 = np.mean(contest_won)
        avg_contest_won_top_5 = np.mean(contest_won[:5])
    else:
        avg_contest_won = np.mean(contest_won)
        avg_contest_won_top_10 = np.mean(contest_won)
        avg_contest_won_top_5 = np.mean(contest_won)

    if len(runner_up) >= 10:
        avg_runner_up = np.mean(runner_up)
        avg_runner_up_top_10 = np.mean(runner_up[:10])
        avg_runner_up_top_5 = np.mean(runner_up[:5])
    elif len(contest_won) >= 5:
        avg_runner_up = np.mean(runner_up)
        avg_runner_up_top_10 = np.mean(runner_up)
        avg_runner_up_top_5 = np.mean(runner_up[:5])
    else:
        avg_runner_up = np.mean(runner_up)
        avg_runner_up_top_10 = np.mean(runner_up)
        avg_runner_up_top_5 = np.mean(runner_up)

    return entry_level, mid_level, top_level, avg_contest_won, avg_contest_won_top_5, avg_contest_won_top_10, avg_runner_up, avg_runner_up_top_5, avg_runner_up_top_10

In [6]:
df_contest[['entry_level', 'mid_level', 'top_level', 'avg_contest_won', 'avg_contest_won_top_5', 'avg_contest_won_top_10', 'avg_runner_up', 'avg_runner_up_top_5', 'avg_runner_up_top_10']] = df_contest.apply(get_user_level, axis=1, result_type="expand")

In [7]:
df_contest[['entry_level', 'mid_level', 'top_level', 'avg_contest_won', 'avg_contest_won_top_5', 'avg_contest_won_top_10', 'avg_runner_up', 'avg_runner_up_top_5', 'avg_runner_up_top_10']].describe()

,entry_level,mid_level,top_level,avg_contest_won,avg_contest_won_top_5,avg_contest_won_top_10,avg_runner_up,avg_runner_up_top_5,avg_runner_up_top_10
count,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000
mean,7.690583,16.578475,3.735426,54.031561,150.706091,104.999859,148.703659,427.092339,295.237216
std,9.018745,15.605441,7.686670,25.133081,73.456377,54.862242,68.964070,228.625285,164.394295
min,0.000000,0.000000,0.000000,7.500000,9.000000,7.500000,11.000000,11.000000,11.000000
25%,3.000000,7.000000,1.000000,36.638258,96.100000,65.325000,100.718750,278.200000,176.825000
50%,5.000000,12.000000,2.000000,49.780423,144.100000,99.700000,134.568548,390.600000,273.000000
75%,10.000000,20.000000,4.000000,64.533333,190.950000,133.800000,183.000000,549.050000,374.725000
max,122.000000,99.000000,80.000000,175.222222,412.600000,353.500000,439.666667,1262.000000,982.000000


In [8]:
df_contest.to_csv("contest_data_merged_" + date + ".csv", index=False)

In [9]:
date_start = "2022-01-20"
date_end = "2022-02-08"

date_range = pd.date_range(start = date_start, end = date_end)

In [10]:
date_range

DatetimeIndex(['2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23',
               '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27',
               '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31',
               '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04',
               '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08'],
              dtype='datetime64[ns]', freq='D')